In [17]:
%run ./get_transactions_history.ipynb
%run ./new_recurrent_income.ipynb

Rows generated successfully
Rows generated successfully


,fecha_de_cargo,descripcion,categoria,monto,forma_de_pago,realizado_por,limite_de_pago,deuda_a_plazos_recurrente_o_normal,comentarios
0,2024-05-17,Credito de nomina,Credito Nomina,500.000000,Crédito Nomina,David,2024-05-17,Plazos,
1,2024-06-02,Credito de nomina,Credito Nomina,500.000000,Crédito Nomina,David,2024-06-02,Plazos,
2,2024-06-17,Credito de nomina,Credito Nomina,500.000000,Crédito Nomina,David,2024-06-17,Plazos,
3,2024-07-02,Credito de nomina,Credito Nomina,500.000000,Crédito Nomina,David,2024-07-02,Plazos,
4,2024-07-17,Credito de nomina,Credito Nomina,500.000000,Crédito Nomina,David,2024-07-17,Plazos,
5,2024-08-02,Credito de nomina,Credito Nomina,500.000000,Crédito Nomina,David,2024-08-02,Plazos,
6,2024-08-17,Credito de nomina,Credito Nomina,500.000000,Crédito Nomina,David,2024-08-17,Plazos,
7,2024-09-02,Credito de nomina,Credito Nomina,500.000000,Crédito Nomina,David,2024-09-02,Plazos,
8,2024-09-17,Credito de nomina,Credito Nomina,500.000000,Crédito Nomina,David,2024-09-17,Plazos,
9,2023-10-29,Celular Samsung A54,Electrónicos,555.555556,LikeU,Aylin,2023-12-01,Plazos,


Stored 'plazos_df' (DataFrame)
Rows generated successfully
Rows generated successfully
Rows generated successfully


,fecha_de_cargo,descripcion,categoria,monto,forma_de_pago,realizado_por,limite_de_pago,deuda_a_plazos_recurrente_o_normal,comentarios
0,2024-05-20,Izzi internet,Servicios,510,Hey Credito,David,2024-07-01,Recurrente,
1,2024-06-20,Izzi internet,Servicios,510,Hey Credito,David,2024-07-31,Recurrente,
2,2024-07-20,Izzi internet,Servicios,510,Hey Credito,David,2024-08-31,Recurrente,
3,2024-08-20,Izzi internet,Servicios,510,Hey Credito,David,2024-10-01,Recurrente,
4,2024-09-20,Izzi internet,Servicios,510,Hey Credito,David,2024-10-31,Recurrente,
5,2024-10-20,Izzi internet,Servicios,510,Hey Credito,David,2024-12-01,Recurrente,
6,2024-11-20,Izzi internet,Servicios,510,Hey Credito,David,2024-12-31,Recurrente,
7,2024-12-20,Izzi internet,Servicios,510,Hey Credito,David,2025-01-31,Recurrente,
8,2025-01-20,Izzi internet,Servicios,510,Hey Credito,David,2025-03-03,Recurrente,
9,2025-02-20,Izzi internet,Servicios,510,Hey Credito,David,2025-03-31,Recurrente,


Stored 'recurrentes_df' (DataFrame)
Stored 'df' (DataFrame)


,freq,fecha_de_ingreso,ingreso_por_evento,cuenta_destino,descripcion,categoria,realizado_por,ingreso_recurrente_o_extraoridinario,comentarios
indice,,,,,,,,,
0,0,2024-03-28,500.00,Efectivo,Vita,Recompensas,David,Recurrente,Ventas
1,1,2024-04-04,500.00,Efectivo,Vita,Recompensas,David,Recurrente,Ventas
2,2,2024-04-11,500.00,Efectivo,Vita,Recompensas,David,Recurrente,Ventas
3,3,2024-04-18,500.00,Efectivo,Vita,Recompensas,David,Recurrente,Ventas
4,4,2024-04-25,500.00,Efectivo,Vita,Recompensas,David,Recurrente,Ventas
5,5,2024-05-02,500.00,Efectivo,Vita,Recompensas,David,Recurrente,Ventas
6,6,2024-05-09,500.00,Efectivo,Vita,Recompensas,David,Recurrente,Ventas
7,7,2024-05-16,500.00,Efectivo,Vita,Recompensas,David,Recurrente,Ventas
8,8,2024-05-23,500.00,Efectivo,Vita,Recompensas,David,Recurrente,Ventas


Stored 'recurrent_incomme_updated' (DataFrame)


In [18]:
from pyspark.sql import SparkSession

In [19]:
spark = SparkSession.builder.getOrCreate()

In [20]:
cargos = spark.read.load("historial_cargos.csv", delimiter = ",", inferSchema=True, header = True, format = "csv")

In [21]:
cargos.show(10)

+--------------+-----------------+--------------+-----+--------------+-------------+-------------+--------------+----------------------------------+
|fecha_de_cargo|      descripcion|     categoria|monto| forma_de_pago|realizado_por|  comentarios|limite_de_pago|deuda_a_plazos_recurrente_o_normal|
+--------------+-----------------+--------------+-----+--------------+-------------+-------------+--------------+----------------------------------+
|    2024-05-10|      Flores Mamá| Regalos Otros|200.0|        Costco|        David|Dia de madres|    2024-06-03|                            Normal|
|    2024-05-09|      Flores Mamá| Regalos Otros|200.0|        Costco|        David|Dia de madres|    2024-06-03|                            Normal|
|    2024-05-17|Credito de nomina|Credito Nomina|500.0|Crédito Nomina|        David|         NULL|    2024-05-17|                            Plazos|
|    2024-06-02|Credito de nomina|Credito Nomina|500.0|Crédito Nomina|        David|         NULL|    2024

In [22]:
cargos.createOrReplaceTempView("cargos")

In [23]:
ingresos = spark.read.load("ingresos_recurrentes_detalle.csv", delimiter = ",", inferSchema=True, header = True, format = "csv")

In [24]:
from pyspark.sql import functions as F

In [25]:
ingresos = ingresos.filter(F.col("categoria") == "Salario")

In [26]:
ingresos.createOrReplaceTempView("ingresos")

In [27]:
spark.sql("""
        select 
            i.fecha_de_ingreso AS lower_salary_date,
            i2.fecha_de_ingreso AS upper_salary_date,
            i.ingreso_por_evento
          FROM ingresos AS i
          LEFT JOIN ingresos AS i2
            ON i.indice = i2.indice - 1
        
          """).createOrReplaceTempView("wage_date_range")

In [28]:
spark.table("wage_date_range").show(30)

+-----------------+-----------------+------------------+
|lower_salary_date|upper_salary_date|ingreso_por_evento|
+-----------------+-----------------+------------------+
|       2024-03-14|       2024-03-29|          22000.12|
|       2024-03-29|       2024-04-14|           23000.0|
|       2024-04-14|       2024-04-29|           23000.0|
|       2024-04-29|       2024-05-14|           23000.0|
|       2024-05-14|       2024-05-29|           23000.0|
|       2024-05-29|       2024-06-14|           23000.0|
|       2024-06-14|       2024-06-29|           23000.0|
|       2024-06-29|       2024-07-14|           23000.0|
|       2024-07-14|       2024-07-29|           23000.0|
|       2024-07-29|       2024-08-14|           23000.0|
|       2024-08-14|       2024-08-29|           23000.0|
|       2024-08-29|       2024-09-14|           23000.0|
|       2024-09-14|       2024-09-29|           23000.0|
|       2024-09-29|       2024-10-14|           23000.0|
|       2024-10-14|       2024-

In [31]:
spark.sql("""
        
        SELECT 
          * 
        FROM cargos AS c
        LEFT JOIN wage_date_range AS dates
          WHERE c.limite_de_pago >= dates.lower_salary_date
          AND c.limite_de_pago < dates.upper_salary_date
          
""").createOrReplaceTempView("gastos_ingresos")

In [33]:
spark.sql("""
        SELECT 
            realizado_por, forma_de_pago, limite_de_pago, lower_salary_date,
            SUM(MONTO) AS Monto_Total
        FROM gastos_ingresos
        GROUP BY
          realizado_por, forma_de_pago, limite_de_pago, lower_salary_date
          """).show(50)

+-------------+--------------+--------------+-----------------+-----------------+
|realizado_por| forma_de_pago|limite_de_pago|lower_salary_date|      Monto_Total|
+-------------+--------------+--------------+-----------------+-----------------+
|        David|      Efectivo|    2024-07-12|       2024-06-29|            120.0|
|        David|   Hey Credito|    2024-10-31|       2024-10-29|           1510.0|
|        Aylin|         LikeU|    2024-10-31|       2024-10-29|555.5555555555555|
|        David|      Efectivo|    2024-08-23|       2024-08-14|            120.0|
|        David|      Efectivo|    2024-06-28|       2024-06-14|            120.0|
|        Aylin|         LikeU|    2024-12-31|       2024-12-29|555.5555555555555|
|        David|Crédito Nomina|    2024-06-17|       2024-06-14|            500.0|
|        Aylin|         LikeU|    2024-07-01|       2024-06-29|555.5555555555555|
|        Aylin|         LikeU|    2024-12-01|       2024-11-29|555.5555555555555|
|        David| 